<h1 align='center'>  Classification of Multiple Classes. </h1>

<p> &nbsp;&nbsp; Sometimes Neural Networks have to predict multiple classes, More then one class solution. There are 2 ways to solve this problem 
<ol>
    <li> Multi Label Classification </li>
    <li> Multi Class Classification </li>
</ol>

<p>&nbsp; 1. Multi Label Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we basically have multiple outputs for multiple classes and get output as 1 or 0 for each neurons output. It invovles one hot encoding process of the dataset </p>

<p>&nbsp; 2. Multi Class Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we use just the single output neuron to predict different classess itself. This involves label encoding process of dataset </p>

<img width="1200" height="800" src='https://www.section.io/engineering-education/multi-label-classification-with-scikit-multilearn/label-powerset.png'>

<p> Let's Work on Multi Class Classification first after which we can move to multi label classification </br>
We Can work with the <b><i> IRIS FLOWER DATASET </i></b>

<h5 align='center'> Dataset: </h5>
<img src='https://miro.medium.com/max/638/0*2c7voFri9cIXGrc4'>

<h3 align='center'> 0. Installing Dependencies </h3>

In [ ]:
!pip install tensorflow
!pip install numpy 
!pip install pandas 
!pip install -U scikit-learn

<h3 align='center'> 1. Importing Dependenceis </h3>

<p> We are going to import all the dependencies for this project. The Dependencies are: </p>
<ul>
    <li> Pandas </li>
    <li> Numpy </li>
    <li>tensorflow </li>
    <li> sklearn </li>
</ul>

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split

<h3 align='center'> 2. Handling Dataset </h3>
<p> &nbsp;&nbsp;&nbsp; We are going to handle the dataset in following ways: </p>
<ul>
    <li> Importing Dataset </li>
    <li> Label encoding </li>
    <li> One hot encoding </li>
</ul>

<h4> Importing Dataset </h4>

In [ ]:
def read_dataset(path):
    data=pd.read_csv(path)
    return data 

def get_class_dict(dataset):
    class_list=list(set(dataset['species'].to_list()))
    class_dct={}
    for i in range(len(class_list)):
        class_dct[class_list[i]]=i
    return class_dct

dataset=read_dataset('IRIS.csv')
classes_dict=get_class_dict(dataset)
#Creating copies of dataset so it can be used to obtain 2 values.
d_temp1=dataset.copy()
d_temp2=dataset.copy()
del dataset #Deleting Dataset freeing some space


<h4> Label Encodng </h4>
<p>&nbsp; &nbsp; &nbsp; We will have each class value in dictionary as label for the class this method of getting out is called label encoding we will use it for multi class classification </p>

In [ ]:
def label_encoding_dataset(d_temp1,class_dict):
    for i in class_dict.keys():
        d_temp1.loc[d_temp1['species']==i,'species']=class_dict[i]
    
    return d_temp1        

dataset_label_encoded=label_encoding_dataset(d_temp1,classes_dict)
dataset_label_encoded



<h4> One Hot Encoding <h4>
<p>&nbsp; &nbsp; &nbsp; Now we are going to have 3 labels each will have 1 or 0 present on it This approach will give us 3 outputs and each output will have 0 or 1 present on it demonstrating yes or no of the output this technique is called one hot encoding and this classification is called Multi Label classification 

In [ ]:
def one_hot_encoding_dataset(dataset,classes_dict):
    def encode(x,i):
        if x==i:
            return 1
        else: 
            return 0

    def add_columns(dataset,classes_dict):
        dataset_final=dataset.drop(columns=['species'])
        for i in classes_dict.keys():
            dataset_final[i]=dataset['species'].apply(encode,args =(i,))
        return dataset_final


    dataset_final=add_columns(dataset,classes_dict)
    return dataset_final

dataset_one_hot_encoded=one_hot_encoding_dataset(d_temp2,classes_dict)
dataset_one_hot_encoded

<h3 align='center'>3. Multi Class Classification </h3>
<p> &nbsp; &nbsp; &nbsp; Multi Class classification will have single output in the neural network that will work out different output values for output neurons.We are going to use softmax for activation function to get the required output.We can see the following image to learn more </p>
<img src='https://miro.medium.com/max/1400/1*ReYpdIZ3ZSAPb2W8cJpkBg.jpeg'>

</br> Our final output neuron of the neural network will be just one consisting of 3 possible values Making our entire architecture of Neural Network as:
</br></br>
<img src='./Diagrams/Multi Class Classification Architecture.jpg'>

<p> The number of neuron assigned to each layer is arbitary at this point but the final layer should have as many neurons as classes to predict because of how softmax activation function works </p>


In [ ]:
def create_model():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid',input_dim=4))
    model.add(tf.keras.layers.Dense(units=18,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=3,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=3,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
    #model.summary()
    return model

def get_X_Y_label_encoded(dataset):
    X=np.array(dataset.drop(columns=['species'])).astype(np.float32)
    Y=(np.array(dataset['species']).astype(np.float32)).reshape(-1,1)
    return X,Y

multi_class_model=create_model()
X,Y=get_X_Y_label_encoded(dataset_label_encoded)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33, random_state=42)
history=multi_class_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5000,batch_size=64)



<h3 align='center'> 3.1 Testing </h3>

We are going to input new custom dataset to see how well it predicts 

In [ ]:
def predict_output(model,input_user):
    output=model.predict([input_user])
    output=np.argmax(output[0])
    output=list({i for i in classes_dict.keys() if classes_dict[i]==output})
    return output[0]

def get_input_from_users(length):
    input_user=[]
    for i in range(1,length+1):
        input_user.append(float(input(f'Enter {i} ipnut ')))
    return input_user
    
input_1=[5.1,3.5,1.4,0.2]
print(predict_output(multi_class_model,input_1))

input_2=get_input_from_users(dataset_label_encoded.shape[1]-1)
print(predict_output(multi_class_model,input_2))
